Adapted tutorial from https://sparkbyexamples.com/spark/spark-read-write-dataframe-parquet-example/

In [5]:
!mkdir /tmp/learn-parquet

mkdir: /tmp/learn-parquet: File exists


In [8]:
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [9]:
# create a sample DF

data = [("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]

rdd = spark.sparkContext.parallelize(data)
df = spark.createDataFrame(rdd, schema=["firstname","middlename","lastname","dob","gender","salary"])
df

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: bigint]

In [15]:
df.write.parquet("/tmp/learn-parquet/people.parquet")

In [16]:
!ls /tmp/learn-parquet/people.parquet

_SUCCESS
part-00000-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet
part-00001-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet
part-00003-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet
part-00005-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet
part-00007-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet
part-00009-11f3938e-a155-42d1-bec0-d1400a35d2b0-c000.snappy.parquet


In [17]:
df = spark.createDataFrame(data, ["firstname","middlename","lastname","dob","gender","salary"])
df.write.parquet("/tmp/learn-parquet/people.parquet2")

In [18]:
!ls /tmp/learn-parquet/people.parquet2

_SUCCESS
part-00000-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
part-00001-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
part-00003-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
part-00005-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
part-00007-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
part-00009-992f69cc-1da5-4acd-9948-c8eca8585e3a-c000.snappy.parquet
